# Neural Network

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import regex as re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
df = pd.read_csv('../anx_writing.csv').drop(columns = ['Unnamed: 0'], axis = 1)

In [4]:
df.head()

,author,link_flair_text,num_comments,subreddit,created_utc,text
0,JackW357,DAE Questions,9,Anxiety,1606687976,Anyone else scared of dying and scared of when...
1,belladoll1021,Health,1,Anxiety,1606687615,Tight throat Can a tight throat and gagging fe...
2,ashwinderegg,Advice Needed,3,Anxiety,1606687588,Anxiety overriding my intuition. Does anyone e...
3,ashwinderegg,Advice Needed,7,Anxiety,1606687588,Anxiety overriding my intuition. Does anyone e...
4,lachapoxxx,Advice Needed,1,Anxiety,1606687488,hey friends! i need some advice my anxiety has...


In [5]:
def custom_preprocessor (text):
    text = text.lower() #lowercases word
    text = re.sub(r'[^\w\s]', '', text) #removes punctuation
    text = re.sub(r'[0–9]', '', text) #removes any numbers
    text = re.sub('(<.*?>)', '', text)
    #copied from https://swatimeena989.medium.com/beginners-guide-for-preprocessing-text-data-f3156bec85ca
    
    return text

#copied from https://www.studytonight.com/post/scikitlearn-countvectorizer-in-nlp

In [6]:
#X = df['text']
#y = df['subreddit']

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7, stratify = y)

In [10]:
# cvec = CountVectorizer(stop_words = 'english', 
#                        strip_accents = 'ascii', 
#                        ngram_range = (1, 2),
#                        preprocessor = custom_preprocessor,
#                        max_features = 50000)   

# X_train_cv = cvec.fit_transform(X_train)

# X_test_cv = cvec.transform(X_test)

In [12]:
# ss = StandardScaler(with_mean=False)
# X_train_sc = ss.fit_transform(X_train_cv)
# X_test_sc = ss.transform(X_test_cv)

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])

In [9]:
max_id = len(tokenizer.word_index)

In [10]:
dataset_size = tokenizer.document_count

6000

[[475]]

In [15]:
def model_fn_deep(hidden_neurons=32, hidden_layers=5, dropout=0.5):
    model = Sequential()
    for layer in range(hidden_layers):
        if layer == 0:
            model.add(Dense(hidden_neurons, input_shape=(), activation='relu'))
            model.add(Dropout(dropout))
        else:
            model.add(Dense(hidden_neurons, activation='relu'))
            model.add(Dropout(dropout))

    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam')

    return model

nn_deep = KerasRegressor(build_fn=model_fn_deep, batch_size=512, verbose=0)

In [16]:
nn_params_deep = {
    'hidden_neurons': [16, 32, 45],
    'hidden_layers': [2, 3, 4, 5],
    'dropout': [0.1, 0.2, 0.5],
    'epochs': [10, 20, 30]
}

gs_deep = GridSearchCV(nn_deep, param_grid=nn_params_deep, cv=3, n_jobs=-1)
gs_deep.fit(X_train_sc, y_train)

/Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: in user code:

    /Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/admin/opt/anaconda3/envs/ten/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/admin/opt/anaconda3/envs/ten

In [13]:
def model_metrics(model, X_train, X_test):
    
    print(f'Best Score: {model.best_score_}')
    print(f'Best Parameters: {model.best_params_}')
    print(f'Train Score: {model.score(X_train, y_train)}')
    print(f'Test Score: {model.score(X_test, y_test)}')

In [ ]:
model_metrics(gs_deep, X_train_sc, X_test_sc)